In [1]:
# Imports for Jupyter
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))
import qgrid
qgrid.enable()
qgrid.set_grid_option('forceFitColumns', False)

# You might have to shutdown and restart ml_experiment.py for the following line to take effect
!jupyter nbextension enable --py --sys-prefix qgrid && jupyter nbextension enable --py --sys-prefix widgetsnbextension
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/sybill/src/src')

Enabling notebook extension qgrid/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import openai
from schema.main import Env
from routers import authentication as api_auth

api_tokens = await api_auth.read_api_secrets(Env.DEV)
openai.organization = api_tokens.openai_organization.get_secret_value()
openai.api_key = api_tokens.openai_api_key.get_secret_value()

In [3]:
import json
from ml.common import general_utils

In [4]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":1000,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()

In [5]:
theme = "dystopian universe"

In [6]:
generatePlotPrompt = """ /* Generate an Plot for a movie

type character{{
  "name": str,
  "background": str
}}

interface overview {{
  "setting": str, //where the section takes place
  "characters": character[], //the character names
  "conflict": str, //what conflict needs to get solved
  "plotTwist": str, //any plot twists to occur
  "resolution": str // how this plot concludes
}}

const generatePlot = (theme) : outline => {{
    return ai.generatePlot(theme)
}}


theme = '{}'
//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(theme))
Out:"""

In [7]:
generateScenesPrompt = """ /* Generate scenes for a movie

type scene{{
  "name": str, // the name of the scene
  "summary": str, // a detailed description with specifics of what happens in this scene
}}

const generateScenes = (plot) : scene[] =>{{
    return ai.generateScenes(plot)
}}

plot = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(plot))
Out:"""

In [8]:
generateDialoguesPrompt = """ /* Write the next set of dialogues in the given scene of the movie

//the overview of the entire movie
movieOverview = {}

//the outline that the movie should follow
movieOutline = {}

//the paragraphs of the movie so far
dialoguesSoFar = {}

interface scene {{
  "dialogues": dialogue[] //all of the dialogues in the story
}}

type dialogue{{
  "name": str, // the name of the speaker
  "text": str, // dialogue that speaker says
}}

//generate multiple longform dialogues for this scene and list them
const {} = (currentScene) : dialogues[] =>{{
    return ai.write()
}}

currentSceneDescription = {}

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + {}(currentOutlineSection))
Out:"""

In [9]:
generateDialoguesPrompt = """ /* Write the next set of dialogues in the movie for the given scene

//the Plot of the entire movie
plot = {}

//the outline that the movie should follow
allScenes = {}

//list of dialogues that have already been spoken
dialoguesSoFar = {}

type dialogue{{
  "name": str, // the name of the speaker
  "text": str, // dialogue that speaker says
}}

//generate multiple longform dialogues for this scene and list them
const {} = (currentScene) : dialogue[] =>{{
    return ai.write()
}}

currentSceneDescription = {}

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + {}(currentOutlineSection))
Out:"""

In [10]:
rawPlot = query(generatePlotPrompt.format(theme), temperature=0.5)
parsedPlot = json.loads(rawPlot)
print('Overall Plot')
display(parsedPlot)

rawScenes = query(generateScenesPrompt.format(rawPlot), temperature=.5)
parsedScenes = json.loads(rawScenes)
print('Overall Scenes')
display(parsedScenes)

dialoguesSoFar = []

for scene in parsedScenes:

    subTheme = f"{scene['name']}: {scene['summary']}\n"
    print(subTheme)

    rawSubPlot = query(generatePlotPrompt.format(subTheme), temperature=0.5)
    parsedSubPlot = json.loads(rawSubPlot)
    print('Sub Plot')
    display(parsedSubPlot)
    
    rawSubScenes = query(generateScenesPrompt.format(rawSubPlot), temperature=.5)
    parsedSubScenes = json.loads(rawSubScenes)
    print('Sub Scenes')
    display(parsedSubScenes)

    for subScene in parsedSubScenes:
        rawDialogues = query(generateDialoguesPrompt.format(rawSubPlot, rawSubScenes, dialoguesSoFar, subScene['name'], subScene['summary'], subScene['name']))
        print(f'Dialogues generated: {len(json.loads(rawDialogues))}')
        dialoguesSoFar += json.loads(rawDialogues)

    for dialogue in dialoguesSoFar:
        print(f"{dialogue['name']}: {dialogue['text']}")
    break

Overall Plot


{'setting': "A dystopian universe where the government has taken control of the people's lives",
 'characters': [{'name': 'John',
   'background': 'An ordinary citizen who is determined to fight against the oppressive government'},
  {'name': 'Jane',
   'background': "John's best friend who is also determined to fight against the oppressive government"},
  {'name': 'The Leader',
   'background': 'The leader of the oppressive government'}],
 'conflict': 'John and Jane must find a way to overthrow the oppressive government and restore freedom to the people',
 'plotTwist': "The Leader is actually John's long lost father",
 'resolution': 'John and Jane are able to overthrow the oppressive government and restore freedom to the people, and John and his father are reunited'}

Overall Scenes


[{'name': 'John and Jane Meet',
  'summary': 'John and Jane meet and discuss their shared desire to fight against the oppressive government. They make a plan to overthrow the government and restore freedom to the people.'},
 {'name': 'John and Jane Take Action',
  'summary': 'John and Jane take action to overthrow the government. They organize protests, rallies, and other forms of civil disobedience to draw attention to the cause. They also recruit other citizens to join their fight.'},
 {'name': "John Discovers the Leader's Identity",
  'summary': 'John discovers that the leader of the oppressive government is actually his long lost father. He is shocked and conflicted, but ultimately decides to continue the fight to restore freedom to the people.'},
 {'name': 'John and Jane Overthrow the Government',
  'summary': 'John and Jane are successful in their efforts and the oppressive government is overthrown. The people are freed and John and his father are reunited.'}]

John and Jane Meet: John and Jane meet and discuss their shared desire to fight against the oppressive government. They make a plan to overthrow the government and restore freedom to the people.

Sub Plot


{'setting': 'A dystopian world ruled by an oppressive government',
 'characters': [{'name': 'John',
   'background': 'A freedom fighter and revolutionary'},
  {'name': 'Jane', 'background': 'A rebel and passionate idealist'}],
 'conflict': 'John and Jane must overthrow the oppressive government to restore freedom to the people.',
 'plotTwist': 'The oppressive government has a secret weapon that could put an end to their plans.',
 'resolution': 'John and Jane rally the people to their cause, defeat the oppressive government, and restore freedom to the people.'}

Sub Scenes


[{'name': 'John and Jane meet',
  'summary': 'John and Jane meet in a secret hideout and discuss their plan to overthrow the oppressive government. They decide to gather a group of freedom fighters to join their cause.'},
 {'name': 'Rally the people',
  'summary': 'John and Jane travel across the country, gathering people to join their cause. They speak to the people about the injustice of the oppressive government and the need for freedom and justice.'},
 {'name': 'The secret weapon',
  'summary': "John and Jane discover the oppressive government's secret weapon, a powerful weapon that could put an end to their plans. They must find a way to disable the weapon or find a way to use it to their advantage."},
 {'name': 'The final battle',
  'summary': 'John and Jane lead their army of freedom fighters into battle against the oppressive government. After a long and hard fought battle, they are victorious and freedom is restored to the people.'}]

Dialogues generated: 5
Dialogues generated: 4


JSONDecodeError: Unterminated string starting at: line 1 column 3439 (char 3438)

In [11]:
for dialogue in dialoguesSoFar:
        print(f"{dialogue['name']}: {dialogue['text']}")

John: We can't let this oppressive government continue to rule over us. We must do something to restore freedom to the people.
Jane: You're right. We need to gather a group of freedom fighters to join our cause and fight for justice.
John: We can start by traveling across the country and speaking to the people about the injustice of the oppressive government.
Jane: Yes, and we can use our words to inspire them to join us in our fight for freedom.
John: Let's do it. We can make a difference and restore freedom to the people.
John: People of this country, we have come to you today to speak of the injustice of the oppressive government. We must stand together and fight for freedom and justice.
Jane: We must not let this government continue to oppress us. We must take a stand and fight for our rights.
John: We need your help to make a difference. Join us in our fight for freedom and justice.
Jane: Together, we can make a difference and restore freedom to the people.
